In [27]:
# load csv files and merge into 1 dataset
import os
import pandas as pd

moralis = pd.DataFrame({}, dtype=str)
filePath = "../data/Moralis/"
for filename in os.listdir(filePath):
    if not filename.endswith('csv'): continue # don't read non .csv files
    filePath_combined = filePath+filename
    newCsv = pd.read_csv(filePath_combined)
    moralis  = moralis.append(newCsv, ignore_index=True)


c:\Users\Thoma\.conda\envs\nftProj\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Users\Thoma\.conda\envs\nftProj\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Users\Thoma\.conda\envs\nftProj\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
moralis.head(1)

,block_number,block_timestamp,block_hash,transaction_hash,transaction_index,log_index,value,contract_type,transaction_type,token_address,token_id,from_address,to_address,amount,verified,operator
0,13296011,2021-09-25T16:15:40.000Z,0x60ec2961f9b979e67e6600db3b32fd2609150926fd81...,0x51636307c3976abb028df5238077c7ca59db4215fd2e...,96,124,30000000000000000,ERC721,Single,0xd3b1ac6d874b8d96f666f25dddddd8d52c5a5c83,1186,0x0000000000000000000000000000000000000000,0x7e3d993a8d63c25d1db5f7ff8a8d6c59091003b1,1,1,NaN


In [22]:
# moralis["token_address"] = moralis["token_address"].str.lower

In [4]:
import pandas as pd
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("../DataOtherSources/EthereumNFTs/nfts.sqlite")

cur = con.cursor()
ethereum = pd.read_sql_query("SELECT * from transfers", con)
# print(df.head(10))
# Be sure to close the connection
con.close()

In [5]:
ethereum["nft_address"] = ethereum["nft_address"].str.lower()

In [12]:
ethereum.head(1)

,transaction_hash,block_number,nft_address,token_id,from_address,to_address,timestamp
0,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629a673a8242c2ac4b7b8c5d8735fbeac21a6205,6682934142305278177002530107138320246486863356...,0x25f1d709b329C7349b8209851E90eFa3a7f60178,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,1627776481


In [10]:
ethereum = ethereum.drop([ 
                "event_id",
                "transaction_value"
                ], axis=1)

In [30]:
moralis.head(1)

,block_number,block_timestamp,transaction_hash,token_address,token_id,from_address,to_address
0,13296011,2021-09-25T16:15:40.000Z,0x51636307c3976abb028df5238077c7ca59db4215fd2e...,0xd3b1ac6d874b8d96f666f25dddddd8d52c5a5c83,1186,0x0000000000000000000000000000000000000000,0x7e3d993a8d63c25d1db5f7ff8a8d6c59091003b1


In [29]:
moralis = moralis.drop([ 
                "block_hash",
                "log_index",
                "value",
                "contract_type",
                "transaction_type",
                "amount",
                "verified",
                "operator",
                "transaction_index"
                ], axis=1)

In [31]:
moralis.columns = ['block_number','timestamp','transaction_hash',"nft_address","token_id","from_address","to_address"]

In [32]:
moralis.head(1)

,block_number,timestamp,transaction_hash,nft_address,token_id,from_address,to_address
0,13296011,2021-09-25T16:15:40.000Z,0x51636307c3976abb028df5238077c7ca59db4215fd2e...,0xd3b1ac6d874b8d96f666f25dddddd8d52c5a5c83,1186,0x0000000000000000000000000000000000000000,0x7e3d993a8d63c25d1db5f7ff8a8d6c59091003b1


In [18]:
ethereum.head(1)

,transaction_hash,block_number,nft_address,token_id,from_address,to_address,timestamp
0,0x0c73daad5e6946e6b2f1374846d6ba3025d6e6a2569f...,12936373,0x629a673a8242c2ac4b7b8c5d8735fbeac21a6205,6682934142305278177002530107138320246486863356...,0x25f1d709b329C7349b8209851E90eFa3a7f60178,0xC69b4c6fFDBaF843A0d0588c99E3C67f27069BEa,1627776481


In [51]:
ethereum['timestamp'] = pd.to_datetime(ethereum['timestamp'],unit='s')
moralis['timestamp'] = pd.to_datetime(moralis['timestamp'])

In [52]:
merge = pd.concat([ethereum,moralis]).reset_index(drop=True)

In [53]:
groupedByNFT = merge.groupby("nft_address")

In [73]:
import numpy as np
import datetime

def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

nft_data_list = []
row = 0
for (nft, groupobject) in groupedByNFT.__iter__():
    nft_data = {}

    nft_data["nft"] = nft
    nft_data["uniqueTokens"] = len(np.unique(groupobject["token_id"]))
    nft_data["uniqueTransactions"] = len(np.unique(groupobject["transaction_hash"]))
    nft_data["uniqueSellers"] = len(np.unique(groupobject["from_address"]))
    nft_data["uniqueBuyers"] = len(np.unique(groupobject["to_address"]))
    nft_data["uniqueMints"] = len(groupobject[groupobject["from_address"]=="0x0000000000000000000000000000000000000000"])

    groupedTokens = groupobject.groupby("token_id")
    tokenLength = []
    tradeLength = []
    for (token, tokenGroup) in groupedTokens.__iter__():
        tokenLength.append(len(tokenGroup))
    
    nft_data["stdTransactionsPerToken"] = np.std(tokenLength)
    nft_data["meanTransactionsPerToken"] = np.mean(tokenLength)
    nft_data["varTransactionsPerToken"] = np.var(tokenLength)

    # nft_data["tradeLength"] = np.var(tradeLength)


    nft_data_list.append(nft_data)

    row =+ 1
    if row == 5: break

nft_meta = pd.DataFrame(nft_data_list)

In [74]:
nft_meta.to_csv("nftMetaData.csv", index=False)